In [2]:
from sentence_transformers import SentenceTransformer
import torch
import spacy
import pandas as pd
import numpy as np
import concurrent.futures
import pickle
import faiss
from transformers import pipeline

nlp = spacy.load("es_core_news_sm")
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(torch.cuda.current_device()))

c:\Users\leand\Documents\tesis\notebooks\preprocesado\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NVIDIA GeForce RTX 2070 SUPER


In [3]:
model_id = "meta-llama/Llama-3.2-1B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    
)

messages = [
    {"role": "system", "content": "Eres una máquina que reformula consultas sobre historia latinoamericana. A cada consulta responderás con un pasaje de texto corto pero relevante que pueda ser útil para responder a la consulta original. No vas a dar respuestas que incluyan títulos, listas o formato Markdown."},
    {"role": "user", "content": "Influencias de Colón en la historia de América"},
]
messages2 = [
    {"role": "system", "content": "Eres una máquina especializada en reformular consultas sobre historia latinoamericana. A cada consulta, tu tarea es hacerla más precisa y clara, sin extenderte demasiado. Solo responderás con la consulta reformulada, sin incluir títulos, listas o formato Markdown."},
    {"role": "user", "content": "Influencias de Colón en la historia de América"},
]
messages3 = [
    {"role": "system", "content": "Eres un experto en historia latinoamericana. Tu tarea es tomar consultas relacionadas con la historia de América Latina y reformularlas para hacerlas más precisas, claras y detalladas. Deberías asegurarte de que la nueva consulta sea más específica, completa y comprensible. Responde solo con la consulta reformulada, sin agregar respuestas completas ni contenido adicional. No uses listas, títulos ni formato Markdown."},
    {"role": "user", "content": "Influencias de Colón en la historia de América"},
]

def get_messages(q):
    return [
        {"role": "system", "content": "Eres un experto en historia latinoamericana. Tu tarea es tomar consultas relacionadas con la historia de América Latina y reformularlas para hacerlas más precisas, claras y detalladas. Deberías asegurarte de que la nueva consulta sea más específica, completa y comprensible. Responde solo con la consulta reformulada, sin agregar respuestas completas ni contenido adicional. No uses listas, títulos ni formato Markdown."},
        {"role": "user", "content": q},
    ]

Device set to use cuda:0


In [9]:
q = "Llegada de Colón a Santo Domingo"

outputs = pipe(
    get_messages(q),
    max_new_tokens=256,
)

new_query = outputs[0]["generated_text"][-1]['content']

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [11]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
def divide_document(text, max_words=300):
    doc = nlp(text)
    sents = [sent.text for sent in doc.sents]
    fragments = []
    temp = ""

    for sent in sents:
        if len(temp.split()) + len(sent.split()) <= max_words:
            temp += " " + sent
        else:
            fragments.append(temp.strip())
            temp = sent

    if temp:
        fragments.append(temp.strip())

    return fragments


In [13]:
df = pd.read_csv("preprocessed.csv")
df

,Path,Text
0,pack\AIBR\2006-Aymaras-peruanos-y-chilenos-en-...,catalogada y caricaturizada bajo el rótulo de ...
1,pack\AIBR\2006-Cambios-de-género-y-discriminac...,agradecimientos la autora desea agradecer a la...
2,pack\AIBR\2006-Consecuencias-personales-en-la-...,has carried out the biggest employment adjustm...
3,pack\AIBR\2006-Construcción-de-modelos-de-géne...,tampoco queremos que se entienda identidad de ...
4,pack\AIBR\2006-Contribuciones-feministas-a-pro...,is irrelevant for others con el que teresa del...
...,...,...
1920,pack\Revista-Española-de-Antropología-American...,"cómo citar: burguete cal y mayor, araceli. 202..."
1921,pack\Revista-Española-de-Antropología-American...,project exposed some critical questionings. in...
1922,pack\Revista-Española-de-Antropología-American...,"1. introducción el interés por los quipus, que..."
1923,pack\Revista-Española-de-Antropología-American...,1. introducción ¿cómo se produce la diferencia...


In [11]:
def divide_and_assign_ids(i, text):
    if type(text) != str:
        return [], []

    frags = divide_document(text)
    return frags, [i] * len(frags)

fragments = []
fragments_ids = []

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    results = list(executor.map(lambda row: divide_and_assign_ids(row[0], row[1]["Text"]),
                                df.iterrows()))

    for frags, ids in results:
        if not frags or not ids:
            continue

        fragments.extend(frags)
        fragments_ids.extend(ids)

len(fragments), len(fragments_ids)

(57716, 57716)

In [14]:
with open("fragments.pkl", "wb") as f:
    pickle.dump((fragments, fragments_ids), f)

In [5]:
with open("fragments.pkl", "rb") as f:
    fragments, fragments_ids = pickle.load(f)

In [16]:
embeddings = model.encode(fragments, convert_to_tensor=True, device=device)
print(embeddings.shape)

torch.Size([57716, 384])


In [17]:
embeddings_np = embeddings.cpu().numpy()
embeddings_np.shape

(57716, 384)

In [19]:
index = faiss.IndexFlatL2(embeddings_np.shape[1])
index.add(embeddings_np)
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x0000023FFF5C7D20> >

In [21]:
faiss.write_index(index, "index-sentence-transformers.faiss")

In [6]:
index = faiss.read_index("index-sentence-transformers.faiss")

In [7]:
def search(query, top_k=10):
    query_embedding = model.encode([query.lower()], convert_to_tensor=True, device=device).cpu().numpy()
    _, indices = index.search(query_embedding, k=top_k*4)

    already_in = set()
    results = []
    for i in indices[0]:
        id = fragments_ids[i]
        if id in already_in:
            continue

        already_in.add(id)
        results.append(id)
        if len(results) == top_k:
            break

    return df.iloc[results]

In [14]:
search(new_query)

,Path,Text
1370,pack\Revista-de-Indias\2020-Aquestos-y-otros-t...,vios y contemporáneos que comprende los fallec...
1160,pack\Revista-de-Indias\2010-La-provincia-de-An...,nistrativo de chile. toda el área que comprend...
828,pack\Anuario-de-Estudios-Americanos\2021-Const...,la red de caminos en chile poco cambió hasta e...
430,pack\Anuario-de-Estudios-Americanos\2001-El-ca...,"por su parte, los ocupantes peruanos del terri..."
831,pack\Anuario-de-Estudios-Americanos\2021-El-Do...,introducción el encuentro entre los dos imperi...
1347,pack\Revista-de-Indias\2019-Colonialismo-depor...,las deportaciones. una política del capitán ge...
1643,pack\Revista-Española-de-Antropología-American...,reaa_40_2_06_lopezperez_maquetación 1 02/07/20...
834,pack\Anuario-de-Estudios-Americanos\2021-Entre...,introducción1 a principios de 1762 era ya evid...
1067,pack\Revista-de-Indias\2006-Poder-y-redes-de-i...,mer asentamiento sería posteriormente enriquec...
863,pack\Anuario-de-Estudios-Americanos\2022-Çiert...,"un aprendiz camino a las indias en 1493, un jo..."
